In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
from collections import ChainMap

### CLEAN UP OF MAIN ABD DATAFRAME

In [2]:
#Load ABD main data

ABD_main = pd.read_csv('ABD_ballToball.csv')

In [3]:
#Dropping unnecessary column
ABD_main.drop(columns = 'Unnamed: 0', inplace = True)

In [4]:
#Appropriate Col name
ABD_main.rename(columns = {'Year' : 'Date', 'Non_Striker' : 'Non Striker'}, inplace = True)

In [5]:
#Convert date string to timestamp
ABD_main['Date'] = ABD_main['Date'].apply(pd.to_datetime)

In [98]:
ABD_main.head()

,Date,Over,Over category,Bowler,Bowler Type,Runs,Extras,Extra Type,Non Striker,Opposition,Venue
0,2017-04-10,1.5,Power Play,Sandeep Sharma,Right-arm fast-medium,4,0,legal,Vishnu Vinod,Punjab Kings,Indore
1,2017-04-10,1.6,Power Play,Sandeep Sharma,Right-arm fast-medium,0,0,legal,Vishnu Vinod,Punjab Kings,Indore
2,2017-04-10,2.4,Power Play,MM Sharma,Right-arm fast-medium,0,0,legal,Vishnu Vinod,Punjab Kings,Indore
3,2017-04-10,2.5,Power Play,MM Sharma,Right-arm fast-medium,0,0,legal,Vishnu Vinod,Punjab Kings,Indore
4,2017-04-10,2.6,Power Play,MM Sharma,Right-arm fast-medium,6,0,legal,Vishnu Vinod,Punjab Kings,Indore


In [7]:
#Changing team names
old_names = ['Deccan Chargers', 'Kings XI Punjab', 'Rising Pune Supergiant', 'Delhi Daredevils']
new_names = ['Sunrisers Hyderabad', 'Punjab Kings', 'Rising Pune Supergiants', 'Delhi Capitals']
for old, new in zip(old_names, new_names):
    ABD_main.loc[ABD_main['Opposition'] == old, 'Opposition'] = new

In [8]:
ABD_main['Opposition'].unique()

array(['Punjab Kings', 'Mumbai Indians', 'Rising Pune Supergiants',
       'Kolkata Knight Riders', 'Gujarat Lions', 'Rajasthan Royals',
       'Delhi Capitals', 'Chennai Super Kings', 'Sunrisers Hyderabad',
       'Kochi Tuskers Kerala', 'Pune Warriors'], dtype=object)

In [9]:
#Load Bowler type
#This was due to having three files and the recapthca problem
#JOB: FIX recapthca
with open('bowler_type_1.json') as f:
    bowler_type1 = json.load(f)
with open('bowler_type_2.json') as f:
    bowler_type2 = json.load(f)
with open('bowler_type_3.json') as f:
    bowler_type3 = json.load(f)
bowler_type = bowler_type1 + bowler_type2 + bowler_type3
#convert list of dicts into main dict
bowler_type_dict = dict(ChainMap(*bowler_type))

In [10]:
#Create a new column for bowler ype using map from bowler
#https://stackoverflow.com/questions/24216425/adding-a-new-pandas-column-with-mapped-value-from-a-dictionary
ABD_main['Bowler Type'] = ABD_main['Bowler'].map(bowler_type_dict)

In [11]:
#convert Overs numeric values into categorical
#https://stackoverflow.com/questions/49382207/how-to-map-numeric-data-into-categories-bins-in-pandas-dataframe

bins = [0, 6, 10, 16, 20]
names = ['Power Play', 'Middle Overs (6-10)', 'Middle Overs (10-16)', 'Death Overs']
ABD_main['Over category'] = pd.cut(ABD_main['Over'], bins, labels=names)

In [12]:
ABD_main.head()

,Over,Bowler,Non Striker,Runs,Extras,Extra Type,Opposition,Venue,Date,Bowler Type,Over category
0,1.5,Sandeep Sharma,Vishnu Vinod,4,0,legal,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
1,1.6,Sandeep Sharma,Vishnu Vinod,0,0,legal,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
2,2.4,MM Sharma,Vishnu Vinod,0,0,legal,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
3,2.5,MM Sharma,Vishnu Vinod,0,0,legal,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
4,2.6,MM Sharma,Vishnu Vinod,6,0,legal,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play


In [13]:
new_cols = ['Date', 'Over', 'Over category', 'Bowler', 'Bowler Type', 'Runs', 'Extras', 'Extra Type', \
            'Non Striker', 'Opposition', 'Venue']
ABD_main = ABD_main[new_cols]

In [14]:
#Manual entry of missing columns for bowler types
#Bowling types of Raiphi gomez, kamran khan, Shalb srivastava, AN ahmed
missing_types = ['Right-arm medium', 'Left-arm fast-medium', 'Left-arm fast-medium', 'Right-arm fast-medium']
for bowler_missing_type, missing_type in \
                    zip(ABD_main.loc[ABD_main['Bowler Type'] == ' ', 'Bowler'].unique(), missing_types):
    ABD_main['Bowler Type'][ABD_main['Bowler'] == bowler_missing_type] = missing_type
#JOB understand the warning   

C:\Users\Owner\Anaconda3\envs\NeuralNets\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Owner\Anaconda3\envs\NeuralNets\lib\site-packages\pandas\core\generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\Owner\Anaconda3\envs\NeuralNets\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
#ABD_main.drop(columns = 'Over', inplace = True)

In [16]:
ABD_main.head()

,Date,Over,Over category,Bowler,Bowler Type,Runs,Extras,Extra Type,Non Striker,Opposition,Venue
0,2017-04-10,1.5,Power Play,Sandeep Sharma,Right-arm fast-medium,4,0,legal,Vishnu Vinod,Punjab Kings,Indore
1,2017-04-10,1.6,Power Play,Sandeep Sharma,Right-arm fast-medium,0,0,legal,Vishnu Vinod,Punjab Kings,Indore
2,2017-04-10,2.4,Power Play,MM Sharma,Right-arm fast-medium,0,0,legal,Vishnu Vinod,Punjab Kings,Indore
3,2017-04-10,2.5,Power Play,MM Sharma,Right-arm fast-medium,0,0,legal,Vishnu Vinod,Punjab Kings,Indore
4,2017-04-10,2.6,Power Play,MM Sharma,Right-arm fast-medium,6,0,legal,Vishnu Vinod,Punjab Kings,Indore


In [17]:
#Goal is prediction of a run based on overs - powerplay, middle overs, death overs (convert overs to categorical)
#                                      bowler_type - available
#                                      previous ball was a no ball? can be obtained easily - free hit means boundary attempt
#                                      bowler_economy previos matches how mnay previous years should I take? featutre problem
#                                      bowler_srike rate - same queston as before
#                                      wckets talen by owler same qestion as previous
#                                      venue - indicator of dimensions
#                                      non striker quality

In [18]:
with open('bowler_stats.json') as f:
    bowler_stats = json.load(f)

In [19]:
bowler_stats_dict = dict(ChainMap(*bowler_stats))

### Creating a data set with last three years + until that match bowler stats and non striker stats

In [218]:
#bowler_stats[0]['Sandeep Sharma'] #- List of years in decreasing order
#bowler_stats[0]['Sandeep Sharma'][0] - Year, Balls, Runs, Wickets, Best figures, Average, Economy, SR etc as keys
#Goal is to create a list (column) that checks for bolwer and the year first and gets prev three years data

ABD_main_V2 = ABD_main.copy()
ABD_main_V2.sort_values(by = 'Date', inplace = True)

In [219]:
#ABD_main_V2.index[0].year
def f(x,y, sub, label):
    #we are accessing row datetime and bowler name
    #scanning through all rows
  
    try:
        #bowler_stats_dict[y] # - has all years of a bowler
        bowler_year_dict = dict(ChainMap(*bowler_stats_dict[x]))
        
    except:
        
        #When there is no stat
        return np.nan
    
    
    #bowler_year_list
    all_years_list = list(bowler_year_dict.keys())
    
    #Pick the key and access the stats
    try:
        
        stat_for_that_year = float(bowler_year_dict[str(y.year- sub)][label])
        
    except:
        
        #when stat was accidently taken from the wrong website
        return np.nan

    return  stat_for_that_year

#https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe 
#ABD_main_V2['Econ_this_year'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 0, 'Economy'), axis = 1)
ABD_main_V2['Economy_Y - 1'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 1, 'Economy'), axis = 1)
ABD_main_V2['Economy_Y - 2'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 2, 'Economy'), axis = 1)
ABD_main_V2['Economy_Y - 3'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 3, 'Economy'), axis = 1)


#Wickets taken
ABD_main_V2['Wickets_Y - 1'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 1, 'Wickets'), axis = 1)
ABD_main_V2['Wickets_Y - 2'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 2, 'Wickets'), axis = 1)
ABD_main_V2['Wickets_Y - 3'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 3, 'Wickets'), axis = 1)

In [220]:
ABD_main_V2.dropna(inplace= True)

In [221]:
ABD_main_V2['3Yr_Econ_Avg'] = round((ABD_main_V2['Economy_Y - 1'] + ABD_main_V2['Economy_Y - 2']\
                                       + ABD_main_V2['Economy_Y - 3']) / 3 , 3)

ABD_main_V2['3Yr_Wickets_Total'] = ABD_main_V2['Wickets_Y - 1'] + ABD_main_V2['Wickets_Y - 2'] \
                                        + ABD_main_V2['Wickets_Y - 3']


In [222]:
ABD_main_V2.drop(columns = ['Economy_Y - 1', 'Economy_Y - 2', 'Economy_Y - 3', \
                           'Wickets_Y - 1', 'Wickets_Y - 2', 'Wickets_Y - 3'], inplace = True )

In [223]:
ABD_main_V2.head()

,Date,Over,Over category,Bowler,Bowler Type,Runs,Extras,Extra Type,Non Striker,Opposition,Venue,3Yr_Econ_Avg,3Yr_Wickets_Total
1194,2011-04-09,13.4,Middle Overs (10-16),R Vinay Kumar,Right-arm fast-medium,1,0,legal,SS Tiwary,Kochi Tuskers Kerala,Kochi,8.287,30.0
1190,2011-04-09,12.2,Middle Overs (10-16),M Muralitharan,Right-arm offbreak,6,0,legal,SS Tiwary,Kochi Tuskers Kerala,Kochi,6.343,40.0
1191,2011-04-09,12.3,Middle Overs (10-16),M Muralitharan,Right-arm offbreak,1,0,legal,SS Tiwary,Kochi Tuskers Kerala,Kochi,6.343,40.0
1192,2011-04-09,13.2,Middle Overs (10-16),R Vinay Kumar,Right-arm fast-medium,0,0,legal,SS Tiwary,Kochi Tuskers Kerala,Kochi,8.287,30.0
1193,2011-04-09,13.3,Middle Overs (10-16),R Vinay Kumar,Right-arm fast-medium,0,0,legal,SS Tiwary,Kochi Tuskers Kerala,Kochi,8.287,30.0


In [224]:
ABD_main_V2.sort_values(by = ['Date', 'Over'], inplace= True)

In [225]:
ABD_main_V2['PreviousBallType'] = ABD_main_V2['Extra Type'].shift(1)

In [226]:
ABD_main_V2.dropna(inplace = True)

In [227]:
ABD_main_V2.drop(columns = 'Extra Type', inplace = True)
ABD_main_V2.head(10)

,Date,Over,Over category,Bowler,Bowler Type,Runs,Extras,Non Striker,Opposition,Venue,3Yr_Econ_Avg,3Yr_Wickets_Total,PreviousBallType
1167,2011-04-09,5.5,Power Play,R Vinay Kumar,Right-arm fast-medium,0,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,8.287,30.0,legal
1168,2011-04-09,5.6,Power Play,R Vinay Kumar,Right-arm fast-medium,0,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,8.287,30.0,legal
1169,2011-04-09,6.4,Middle Overs (6-10),M Muralitharan,Right-arm offbreak,2,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,6.343,40.0,legal
1170,2011-04-09,6.5,Middle Overs (6-10),M Muralitharan,Right-arm offbreak,0,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,6.343,40.0,legal
1171,2011-04-09,6.6,Middle Overs (6-10),M Muralitharan,Right-arm offbreak,0,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,6.343,40.0,legal
1172,2011-04-09,7.4,Middle Overs (6-10),R Vinay Kumar,Right-arm fast-medium,1,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,8.287,30.0,legal
1173,2011-04-09,8.1,Middle Overs (6-10),S Sreesanth,Right-arm fast-medium,1,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,9.007,28.0,legal
1174,2011-04-09,8.3,Middle Overs (6-10),S Sreesanth,Right-arm fast-medium,1,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,9.007,28.0,legal
1175,2011-04-09,8.5,Middle Overs (6-10),S Sreesanth,Right-arm fast-medium,1,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,9.007,28.0,legal
1176,2011-04-09,8.7,Middle Overs (6-10),S Sreesanth,Right-arm fast-medium,6,0,MA Agarwal,Kochi Tuskers Kerala,Kochi,9.007,28.0,legal


In [232]:
#Rearrange columns
new_cols = ['Date', 'Over category', 'Bowler', 'Bowler Type', '3Yr_Econ_Avg', '3Yr_Wickets_Total' \
                , 'PreviousBallType', 'Opposition', 'Venue', 'Runs' ]
ABD_main_V2 = ABD_main_V2[new_cols]

In [233]:
ABD_main_V2.head()

,Date,Over category,Bowler,Bowler Type,3Yr_Econ_Avg,3Yr_Wickets_Total,PreviousBallType,Opposition,Venue,Runs
1167,2011-04-09,Power Play,R Vinay Kumar,Right-arm fast-medium,8.287,30.0,legal,Kochi Tuskers Kerala,Kochi,0
1168,2011-04-09,Power Play,R Vinay Kumar,Right-arm fast-medium,8.287,30.0,legal,Kochi Tuskers Kerala,Kochi,0
1169,2011-04-09,Middle Overs (6-10),M Muralitharan,Right-arm offbreak,6.343,40.0,legal,Kochi Tuskers Kerala,Kochi,2
1170,2011-04-09,Middle Overs (6-10),M Muralitharan,Right-arm offbreak,6.343,40.0,legal,Kochi Tuskers Kerala,Kochi,0
1171,2011-04-09,Middle Overs (6-10),M Muralitharan,Right-arm offbreak,6.343,40.0,legal,Kochi Tuskers Kerala,Kochi,0


In [185]:
# NON STRIKER STATS
#with open('non_striker_stat.json') as f:
#    non_striker_stats = json.load(f)
#non_striker_stats_dict = dict(ChainMap(*non_striker_stats))

In [246]:
ABD_main_V2.to_csv('RUns_ThreeYearAvgWickets.csv')